# Расчёт токов короткого замыкания согласно IEC 60909

pandapower поддерживает расчёт токов короткого замыкания методом эквивалентного источника напряжения в точке повреждения в соответствии со стандартом IEC 60909. Расчёт коротких замыканий в pandapower учитывает следующие элементы:

- sgen (в качестве двигателя или генератора с полным преобразователем)
- gen (в качестве синхронного генератора)
- ext_grid (внешняя сеть)
- line (линия)
- trafo (двухобмоточный трансформатор)
- trafo3w (трёхобмоточный трансформатор)
- impedance (импеданс)

с применением поправочных коэффициентов, определённых в IEC 60909. Нагрузки и шунты, согласно стандарту, не учитываются. Модель коммутационных аппаратов (switch) pandapower полностью интегрирована в расчёт короткого замыкания.

Могут быть рассчитаны следующие токи короткого замыкания:
- ikss (начальный симметричный ток короткого замыкания)
- ip (ударный ток короткого замыкания)
- ith (эквивалентный термический ток короткого замыкания)

как при

- симметричном трёхфазном, так и при
- несимметричном двухфазном

коротком замыкании. Расчёты применимы как для замкнутых (сетевых), так и для радиальных сетей. Величины ip и ith реализованы только для случаев короткого замыкания, удалённых от синхронных генераторов.

Результаты расчётов для всех элементов и различных токов короткого замыкания проверены путём сравнения с коммерческим программным обеспечением, чтобы гарантировать корректное применение поправочных коэффициентов.

### Пример сети

Вот небольшой пример использования расчёта короткого замыкания. Сначала создаём простую разомкнутую кольцевую сеть из 4 узлов (шин), соединённых одним трансформатором и двумя линиями, причём одна секционирующая точка разомкнута. Питание сети осуществляется от внешней сети (ext_grid) в узле 1:

<img src="shortcircuit/example_network_sc.png">

In [ ]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)
    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

## Расчёт симметричных токов короткого замыкания

### Максимальные токи короткого замыкания
Теперь загружаем сеть и вычисляем максимальные токи короткого замыкания с помощью функции `calc_sc`:

In [13]:
net = ring_network()
sc.calc_sc(net, case="max", ip=True, ith=True, branch_results=True)
net.res_bus_sc

Branch results are in beta mode and might not always be reliable, especially for transformers
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bool)
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bo

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.262432,100.000000,0.505834,0.263723,175.878566,502.510189
1,0.476454,90.776637,0.942589,0.479039,44.276323,139.778739
2,0.466671,88.912758,0.915418,0.469123,46.571323,142.268739
3,0.469892,89.526397,0.924301,0.472386,45.806323,141.438739


где **ikss** — начальный ток короткого замыкания, **ip** — ударный (пиковый) ток короткого замыкания, а **ith** — эквивалентный термический ток.

Для ветвей результаты определяются как максимальные токи, протекающие по ветви при возникновении короткого замыкания в любом из узлов сети. Результаты доступны отдельно для линий:

In [14]:
net.res_line_sc

,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.466671,0.466671,-71.874268,0.466671,108.125732,1.499428,1.62683,0.000000,0.000000,0.024883,-24.540621,0.016703,-24.721299,0.915418,0.469123
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.016703,-24.721299,0.016703,-24.721299,0.000000,0.000000
2,0.469892,0.469892,107.945055,0.469892,-72.054945,0.000000,0.00000,1.013464,1.099575,0.024883,-24.540621,0.024883,-24.540621,0.924301,0.472386


и для трансформаторов:

In [15]:
net.res_trafo_sc

,ikss_hv_ka,ikss_hv_degree,ikss_lv_ka,ikss_lv_degree,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree
0,0.238227,-72.423878,0.476454,107.576122,1.699796,10.872445,0.0,0.0,0.123768,9.240049,0.024883,-24.540621


### Минимальные токи короткого замыкания

Минимальные токи короткого замыкания можно рассчитать аналогичным образом. Однако сначала необходимо задать, согласно стандарту, конечную температуру проводов после повреждения:

In [16]:
net = ring_network()
net.line["endtemp_degree"] = 80
sc.calc_sc(net, case="min", ith=True, ip=True, branch_results=True)
net.res_bus_sc

Branch results are in beta mode and might not always be reliable, especially for transformers
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bool)
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bo

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.209946,80.000000,0.462534,0.211736,118.650262,593.251309
1,0.384422,73.242307,0.860874,0.387974,29.969247,162.464019
2,0.377608,71.943998,0.832832,0.380846,32.815047,164.954019
3,0.379861,72.373180,0.841982,0.383197,31.866447,164.124019


Теперь результаты для ветвей представляют собой минимальные токи, протекающие через каждую ветвь:

In [17]:
net.res_line_sc

,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.377608,0.377608,-78.748780,0.377608,101.251220,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,8.328324e-01,3.808460e-01
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,1.000000e+10,1.000000e+10
2,0.379861,0.379861,100.987879,0.379861,-79.012121,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,8.419815e-01,3.831970e-01


In [18]:
net.res_trafo_sc

,ikss_hv_ka,ikss_hv_degree,ikss_lv_ka,ikss_lv_degree,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree
0,0.188804,-78.74878,0.377608,101.25122,0.0,0.0,-1.217329,-1.065131,0.0,0.0,0.0,0.0


### Асинхронные двигатели

Асинхронные двигатели задаются путём создания статического генератора (static generator) типа «motor». Для расчёта сопротивления при коротком замыкании необходимо указать отношение R/X (параметр «rx»), а также коэффициент «k» — отношение номинального тока двигателя к его току короткого замыкания:

In [19]:
net = ring_network()
pp.create_sgen(net, 2, p_mw=0, sn_mva=0.5, k=1.2, rx=7., type="motor")
net

This pandapower network includes the following parameter tables:
   - bus (4 elements)
   - sgen (1 element)
   - switch (1 element)
   - ext_grid (1 element)
   - line (3 elements)
   - trafo (1 element)

Если мы снова выполним расчёт короткого замыкания, то увидим, что токи увеличились за счёт вклада инвертеров в токи короткого замыкания.

In [20]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bool)
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bool)
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\shortc

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.264007,100.600000,0.508061,0.265306,175.878566,502.510189
1,0.479603,91.376637,0.947043,0.482205,44.276323,139.778739
2,0.469820,89.512758,0.919871,0.472289,46.571323,142.268739
3,0.472998,90.118134,0.928693,0.475509,45.806323,141.438739


### Синхронные генераторы

Синхронные генераторы также могут учитываться при расчёте короткого замыкания с использованием элемента `gen`. Согласно стандарту, для определения сопротивления при коротком замыкании необходимо задать следующие параметры: номинальный коэффициент мощности (cos φ) — «cos_phi», номинальное напряжение — «vn_kv», номинальную полную мощность — «sn_kva», а также сверхпереходное активное сопротивление «rdss» и сверхпереходное реактивное сопротивление «xdss»:

In [21]:
net = ring_network()
pp.create_gen(net, 2, p_mw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_mva=5, xdss_pu=0.2, rdss_pu=0.005)
net

This pandapower network includes the following parameter tables:
   - bus (4 elements)
   - gen (1 element)
   - switch (1 element)
   - ext_grid (1 element)
   - line (3 elements)
   - trafo (1 element)

и снова выполним расчёт короткого замыкания:

In [22]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

aperiodic, thermal short-circuit currents are only implemented for faults far from generators!
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(bool)
c:\Users\sasha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandapower\build_branch.py:1342: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  power_station_unit = trafo_df.power_station_unit.fillna(False).values.astype(b

AttributeError: 'DataFrame' object has no attribute 'rdss_ohm'

Опять же, ток короткого замыкания увеличивается за счёт вклада генератора. Как указано в предупреждении, значения ударного тока (ip) и эквивалентного термического тока короткого замыкания (ith) будут точными только для повреждений, удалённых от генераторов.

## Замкнутые (сетевые) сети

Поправочные коэффициенты для апериодической составляющей и термического тока различаются для замкнутых (сетевых) и радиальных сетей. pandapower включает автоматическое определение типа топологии, которое для каждого места короткого замыкания самостоятельно выявляет, является ли сеть замкнутой или радиальной. Кроме того, тип топологии можно явно задать как «radial» (радиальная) или «meshed» (замкнутая), чтобы обойти автоматическую проверку и сократить время расчёта.

Загружаем радиальную сеть и замыкаем ранее разомкнутую секционирующую точку, чтобы получить замкнутую кольцевую сеть:

In [ ]:
net = ring_network()
net.switch.closed = True
sc.calc_sc(net, topology="auto", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.470593,0.926244,0.473098
3,0.471649,0.929174,0.474168


Сеть автоматически определяется как замкнутая (meshed), и соответствующие поправочные коэффициенты применяются автоматически. Это можно проверить, явно задав топологию как радиальную («radial») и сравнив полученные результаты:

In [ ]:
sc.calc_sc(net, topology="radial", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.470593,0.926244,0.473098
3,0.471649,0.929174,0.474168


Если посмотреть на результаты для линий, можно увидеть, что токи в линиях значительно меньше токов в узлах (шинах):

In [ ]:
sc.calc_sc(net, topology="auto", ip=True, ith=True, branch_results=True)
net.res_line_sc

tazan.pandapower.shortcircuit.calc_sc - WARNING: Branch results are in beta mode and might not always be reliable, especially for transformers


,ikss_ka,ip_ka,ith_ka
0,0.279812,0.550740,0.281301
1,0.190781,0.375504,0.191796
2,0.344176,0.678046,0.346014


это происходит потому, что ток короткого замыкания распределяется по обоим путям кольца, что корректно учитывается pandapower.

## Сопротивление в месте повреждения

В расчёте короткого замыкания также можно задать сопротивление в месте повреждения:

In [ ]:
net = ring_network()
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

что, конечно же, приводит к уменьшению токов короткого замыкания:

In [ ]:
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.261343,0.503509,0.262627
1,0.469382,0.926656,0.471909
2,0.459875,0.900379,0.462274
3,0.463005,0.908972,0.465446


## Расчёт несимметричного двухфазного тока короткого замыкания

Все приведённые выше расчёты могут быть выполнены и для двухфазного тока короткого замыкания аналогичным образом — достаточно указать параметр `fault="2ph"`:

In [ ]:
net = ring_network()
sc.calc_sc(net, fault="2ph", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.227273,0.438065,0.228391
1,0.412621,0.816306,0.414860
2,0.404149,0.792775,0.406272
3,0.406938,0.800468,0.409099


Двухфазные короткие замыкания часто используются при расчётах минимальных токов короткого замыкания:

In [ ]:
net = ring_network()
net.line["endtemp_degree"] = 150
sc.calc_sc(net, fault="2ph", case="min", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.181818,0.400566,0.183369
1,0.332920,0.745538,0.335995
2,0.326772,0.717815,0.329521
3,0.328807,0.726834,0.331657


## Расчёт однофазного тока короткого замыкания

pandapower также позволяет рассчитывать однофазные токи короткого замыкания. Однако замыкание на землю зависит от параметров нулевой последовательности сети, которые необходимо задать для выполнения расчёта однофазного замыкания на землю:

In [ ]:
net = ring_network()

#r/x ratio in zero sequence parameters
net.ext_grid["r0x0_max"] = 0.4
net.ext_grid["x0x_max"] = 1.0

#zero sequence line parameters
net.line["r0_ohm_per_km"] = 0.244
net.line["x0_ohm_per_km"] = 0.336
net.line["c0_nf_per_km"] = 2000

#transformer vector group, zero sequence short circuit voltage
#and zero sequence magnetizing impedance
net.trafo["vector_group"] = "Dyn"
net.trafo["vk0_percent"] = 5.
net.trafo["vkr0_percent"] = 0.4
net.trafo["mag0_percent"] = 10
net.trafo["mag0_rx"] = 0.4
net.trafo["si0_hv_partial"] = 0.9

In [ ]:
sc.calc_sc(net, fault="1ph")

In [ ]:
net.res_bus_sc

,ikss_ka
0,0.261047
1,0.698664
2,0.668867
3,0.680010
